What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [1]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [2]:
# Read in postcode dataset
UU_postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU Zonal nitrate for imperial final 2000-2010_ed.csv")

In [ ]:
UU_postcode

- Get rid of the white space in the postcodes that have XXXX XXX to XXXXXXX characters so that they match with the OS codepoint dataset

In [4]:
UU_postcode['Postcode'] = UU_postcode['Postcode'].str.strip()

UU_postcode.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_test.csv")

In [5]:
mask = (UU_postcode['Postcode'].str.len()>7)
mask2 = (UU_postcode['Postcode'].str.len()<8)

In [6]:
df1 = UU_postcode.loc[mask]

In [7]:
df2 = UU_postcode.loc[mask2]

In [ ]:
df1

In [ ]:
df2

In [10]:
df1['First']= df1['Postcode'].str[:4]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df1['Last']= df1['Postcode'].str[5:8]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1['New_pstcd'] = df1['First'].map(str) + df1['Last']

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1

df1.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_df1.csv")

In [14]:
df1_n = df1[['Date', 'New_pstcd', 'Nitrate', 'Qualifier', 'Zone desc']]

In [15]:
df1_n = df1_n.rename(columns={'New_pstcd':'Postcode'})

In [16]:
df_all = pd.concat([df1_n, df2])

In [17]:
df_all = df_all.set_index('Postcode')

In [ ]:
df_all

- read in the header file for the Code point postcode files 
- set the column headings to the first row (which contains the header names)
- set the index to the column which says postcode
- delete the row with the duplicate of column headings- so end up with a table of just the column headings

In [19]:
labels2 = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [20]:
labels2.columns = labels2.iloc[0]

In [21]:
labels2

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [22]:
labels2 = labels2.set_index('Postcode')

In [23]:
labels2 = labels2.reindex(labels2.index.drop('Postcode'))

- merge the SWW postcodes and the labels to create a dataframe in which we can fill with the OS easting and northing information

In [24]:
df_all = df_all.merge(labels2, how='left', left_index=True, right_index=True)

In [ ]:
df_all

- read in the labels data again so that the OS codepoint files can be merged with the column headings

In [26]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

- Create a list of the files within the codepoint folder (containing all the different postcodes) to use within the for loop

In [27]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\*")

In [28]:
files

['S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\tw.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dy.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\la.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\hd.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\sl.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dg.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ph.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ws.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\e.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\gu.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ig.csv',


- create a loop to insert data into the df_all dataset
    - read the postcode file in
    - merge the labels and postcode file so the postcode file has column headings 
    - set the column headings as actual column headings 
    - set the index to postcode
    - delete the duplicate postcode row
    - insert a column 'date' because the table we are filling has a date column
    - ensure that the date column is in datetime format
    - insert the postcode information into the SWW postcode dataset
    - delete all of the extra postcodes that aren't in the SWW postcode dataset
    - display that the postcode file has been processed 

In [30]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'Date', 'NaN')
    data.insert(1,'Nitrate', 'NaN')
    data.insert(2,'Qualifier', 'NaN')
    data.insert(3,'Zone desc', 'NaN')
    data['Date']= pd.to_datetime(data['Date'])
    df_all = df_all.combine_first(data)
    df_all = df_all[pd.notnull(df_all['Date'])]
    
    
    print('Processed {fname}'.format(fname=file))

Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\tw.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dy.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\la.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\hd.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\sl.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dg.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ph.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ws.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\e.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\gu.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ig.csv
Processed S:\Projects\SAHSU_Share

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

- save the dataset

In [31]:
df_all.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_Northing_Easting.csv")

- rerunning the code on the dataset where O's and I's have been changed to 0's and 1's

In [32]:
# Read in postcode dataset
post_data = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_Unmatched.csv")

In [33]:
post_data = post_data.set_index('Postcode')

In [38]:
labels

,0,1,2,3,4,5,6,7,8,9
0,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [ ]:
post_data

In [35]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\*")

In [36]:
files

['S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\tw.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dy.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\la.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\hd.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\sl.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dg.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ph.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ws.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\e.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\gu.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ig.csv',


In [37]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [39]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'date', 'NaN')
    data.insert(1,'Nitrate', 'NaN')
    data.insert(2,'Qualifier', 'NaN')
    data.insert(3,'Zone desc', 'NaN')
    data['date']= pd.to_datetime(data['date'])
    post_data = post_data.combine_first(data)
    post_data = post_data[pd.notnull(post_data['date'])]
    
    
    print('Processed {fname}'.format(fname=file))

Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\tw.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dy.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\la.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\hd.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\sl.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dg.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ph.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ws.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\e.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\gu.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ig.csv
Processed S:\Projects\SAHSU_Share

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [40]:
post_data

,Admin_county_code,Admin_district_code,Admin_ward_code,Country_code,Date,Eastings,NHS_HA_code,NHS_regional_HA_code,Nitrate,Northings,Positional_quality_indicator,Qualifier,Zone desc,date
Postcode,,,,,,,,,,,,,,


In [ ]:
post_data_c = post_data[[ 'Eastings', 'Nitrate', 'Northings', 'date']]

In [ ]:
post_data_c

In [ ]:
east_northing = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\SWW Nitrate 2011-PRESENT_northing_easting.csv")

In [ ]:
east_northing

In [ ]:
east_northing = east_northing.set_index(['Postcode'])

In [ ]:
all_SWW = pd.concat([post_data_c, east_northing])

In [ ]:
all_SWW

In [ ]:
all_SWW.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW.csv')

- read in unmatched SWW data 
- match with the postcode district centroid files
- match with postcode csv

In [ ]:
unmatched = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW_unmatched.csv")

In [ ]:
unmatched

In [ ]:
unmatched = unmatched.set_index(['Postcode_editted'])

In [ ]:
postcode_dist = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\Post_d_centroids.csv")

In [ ]:
postcode_dist

In [ ]:
postcode_dist = postcode_dist.rename(columns={'X_Coord':'Eastings', 'Y_Coord':'Northings','District':'Postcode_editted'})

In [ ]:
postcode_dist = postcode_dist.set_index(['Postcode_editted'])

In [ ]:
postcode_dist = postcode_dist[['Eastings', 'Northings']]

In [ ]:
postcode_dist.insert(0, 'Postcode', 'NaN')

In [ ]:
postcode_dist.insert(2, 'Nitrate', 'NaN')
postcode_dist.insert(4, 'date', 'NaN')

In [ ]:
postcode_dist['date']= pd.to_datetime(postcode_dist['date'])

In [ ]:
unmatched = unmatched.combine_first(postcode_dist)
unmatched = unmatched[pd.notnull(unmatched['date'])]

In [ ]:
unmatched

In [ ]:
postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\all_postcodes_cut.csv")

In [ ]:
postcode

In [ ]:
postcode = postcode.rename(columns={'Postcode':'Postcode_editted'})

In [ ]:
postcode = postcode.set_index(['Postcode_editted'])

In [ ]:
postcode.insert(0, 'Postcode', 'NaN')

In [ ]:
postcode.insert(2, 'Nitrate', 'NaN')
postcode.insert(4, 'date', 'NaN')

In [ ]:
postcode['date']= pd.to_datetime(postcode['date'])

In [ ]:
unmatched = unmatched.combine_first(postcode)
unmatched = unmatched[pd.notnull(unmatched['date'])]

In [ ]:
unmatched

In [ ]:
unmatched.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW_unmatched_matched.csv')